# This is RFdiffusion finetuning


In [1]:
import torch

# Load the checkpoint
checkpoint = torch.load("/home/yunyao/RFdiffusion/models/Base_epoch8_ckpt.pt")

In [30]:
print(checkpoint.keys())
# Extract the model state dictionary
model_state_dict = checkpoint['model_state_dict']
# Print the keys of the state dictionary        
print(checkpoint['config_dict'].keys())
print(checkpoint['config_dict']['model'].keys())
print(checkpoint['config_dict']['preprocess'].keys())

dict_keys(['model_state_dict', 'final_state_dict', 'config_dict'])
dict_keys(['model', 'diffuser', 'seq_diffuser', 'preprocess'])
dict_keys(['n_extra_block', 'n_main_block', 'n_ref_block', 'd_msa', 'd_msa_full', 'd_pair', 'd_templ', 'n_head_msa', 'n_head_pair', 'n_head_templ', 'd_hidden', 'd_hidden_templ', 'p_drop', 'd_time_emb', 'd_time_emb_proj', 'freeze_track_motif', 'use_motif_timestep', 'SE3_param_full', 'SE3_param_topk'])
dict_keys(['sidechain_input', 'motif_sidechain_input', 'sequence_decode', 'd_t1d', 'd_t2d', 'predict_previous', 'prob_self_cond', 'str_self_cond', 'seq_self_cond', 'new_self_cond'])


In [24]:
valid_keys = signature(RoseTTAFoldModule.__init__).parameters.keys()
print(valid_keys)

odict_keys(['self', 'n_extra_block', 'n_main_block', 'n_ref_block', 'd_msa', 'd_msa_full', 'd_pair', 'd_templ', 'n_head_msa', 'n_head_pair', 'n_head_templ', 'd_hidden', 'd_hidden_templ', 'p_drop', 'd_t1d', 'd_t2d', 'T', 'use_motif_timestep', 'freeze_track_motif', 'SE3_param_full', 'SE3_param_topk', 'input_seq_onehot'])


In [31]:
from rfdiffusion.RoseTTAFoldModel import RoseTTAFoldModule
from inspect import signature

valid_keys = signature(RoseTTAFoldModule.__init__).parameters.keys()
filtered_model_config = {}

# Iterate through all keys in checkpoint['config_dict']
for key in checkpoint['config_dict']:
    # Ensure the value is a dictionary to avoid issues
    if isinstance(checkpoint['config_dict'][key], dict):
        # Filter valid parameters for the current key
        filtered_model_config.update(
            {k: v for k, v in checkpoint['config_dict'][key].items() if k in valid_keys}
        )

# Initialize the model
model = RoseTTAFoldModule(**filtered_model_config)

# Load the model state dictionary into the model
model.load_state_dict(model_state_dict, strict=False)

<All keys matched successfully>

In [32]:
print(model)

RoseTTAFoldModule(
  (latent_emb): MSA_emb(
    (emb): Linear(in_features=48, out_features=256, bias=True)
    (emb_q): Embedding(22, 256)
    (emb_left): Embedding(22, 128)
    (emb_right): Embedding(22, 128)
    (emb_state): Embedding(22, 64)
    (drop): Dropout(p=0.15, inplace=False)
    (pos): PositionalEncoding2D(
      (emb): Embedding(65, 128)
      (drop): Dropout(p=0.15, inplace=False)
    )
  )
  (full_emb): Extra_emb(
    (emb): Linear(in_features=25, out_features=64, bias=True)
    (emb_q): Embedding(22, 64)
    (drop): Dropout(p=0.15, inplace=False)
  )
  (templ_emb): Templ_emb(
    (emb): Linear(in_features=88, out_features=64, bias=True)
    (templ_stack): TemplatePairStack(
      (block): ModuleList(
        (0-1): 2 x PairStr2Pair(
          (emb_rbf): Linear(in_features=36, out_features=32, bias=True)
          (proj_rbf): Linear(in_features=32, out_features=64, bias=True)
          (drop_row): Dropout()
          (drop_col): Dropout()
          (row_attn): BiasedAxia

In [ ]:
# now let us freeze certain layers
